In [24]:
#Note Run receiver code in one
#    desktop and then this sender code need to be run in another device with proper host and port label which can 
#    secure the connection between the two devices and send data properly.
"""
    Protocol procedure at the sending end as described in USLP section 4.2 
    MAP Packet Service (MAP Packet Processing) and MAP Access Service (MAPA_SDU Generation)
    have been implemented here and the code given for sender. 
    
    The MAP Packet Processing Function is crucial for handling the multiplexing of data packets onto the physical link. 
    It involves segmenting higher-level data packets into smaller packets suitable for transmission, 
    potentially adding headers for identification, and managing the transmission order.
    
     The MAPA_SDU (Service Data Unit) is essentially the data unit that the MAP accepts for processing, 
     potentially including additional information for managing transmission, such as Quality of Service (QoS) parameters.
    
    """

def map_packet_processing(data, max_payload_size=48, qos=1):
    """
    MAP Packet Processing Function by segmenting data into MAPA_SDUs, each with a QoS byte.
    :param max_payload_size: The maximum payload size for each MAPA_SDU.
    :param qos: Quality of Service byte (simplified to a single byte for priority).
    :return: A list of MAPA_SDUs ready for transmission.
    """
    mapas = []
    for i in range(0, len(data), max_payload_size):
        segment = data[i:i+max_payload_size]
        packet_length = len(segment) + 1  # +1 for the QoS byte
        header = packet_length.to_bytes(2, byteorder='big')
        mapas.append(header + qos.to_bytes(1, byteorder='big') + segment.encode('utf-8'))
    return mapas

import socket

def send_data(data, host='localhost', port=12345, qos=1):
    packets = map_packet_processing(data, qos=qos)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((host, port))
        for packet in packets:
            s.sendall(packet)
            print(f"Sent packet with QoS {qos}: {packet}")

if __name__ == "__main__":
    send_data("Hi. This message will be segmented into smaller packets according to the MAP processing function from USLP frame.")



Sent packet with QoS 1: b'\x001\x01Hi. This message will be segmented into smaller '
Sent packet with QoS 1: b'\x001\x01packets according to the MAP processing function'
Sent packet with QoS 1: b'\x00\x12\x01 from USLP frame.'


In [25]:
"""
    USLP TRANSFER FRAME as described in USLP section 4.1
    A USLP Transfer Frame shall encompass the major fields, positioned contiguously, in the following sequence:
a) Transfer Frame Primary Header (4 to 14 octets, mandatory);
b) Transfer Frame Insert Zone (integral number of octets, optional);
c) Transfer Frame Data Field (integral number of octets, mandatory);
d) Operational Control Field (4 octets, optional); and
e) Frame Error Control Field (2 octets, optional).
Here I implemented part a,c,e in this code for simplicity and the frame is checked accordingly.
This is the sender code which need to be implemented in one
    desktop and the receiver code need to be run in another device with proper host and port label which can 
    secure the connection between the two devices and send data properly.
    """
import socket
import crcmod
import struct

# CRC-16-CCITT function
crc16_func = crcmod.predefined.mkPredefinedCrcFun('crc-ccitt-false')
def create_primary_header(frame_count, version_number=0, spacecraft_id=0, virtual_channel_id=0):
    """
    Creates a simple USLP Primary Header.
    :param version_number: Version of the USLP protocol.
    :param spacecraft_id: Identifier for the spacecraft.
    :param virtual_channel_id: Virtual channel identifier.
    """
    # Simplified example: 4 bytes total for demonstration
    # Adjust sizes and bit packing as per your specific protocol details
    header_bytes = struct.pack('>BBBB', version_number, spacecraft_id, virtual_channel_id, frame_count)
    return header_bytes
def create_uslp_frame(data, frame_count):
    primary_header = create_primary_header(frame_count)
    data_field = data  # Assuming data is already in bytes format
    # Calculate CRC (FECF) over header and data field
    crc_input = primary_header + data_field
    crc_value = crc16_func(crc_input)
    # Append CRC as FECF (2 bytes)
    fecf = struct.pack('>H', crc_value)
    return primary_header + data_field + fecf

def send_uslp_frame(data, host='localhost', port=12345, qos=1):
    packets = map_packet_processing(data, qos=qos)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((host, port))
        frame_count = 1  # Example frame count
        uslp_frame = create_uslp_frame(data.encode('utf-8'), frame_count)
        sock.sendall(uslp_frame)
        print("Sent USLP Frame:", uslp_frame.hex())

if __name__ == "__main__":
    send_uslp_frame("This is a longer message that will be segmented into smaller packets according to the MAP processing function.")


Sent USLP Frame: 00000001546869732069732061206c6f6e676572206d65737361676520746861742077696c6c206265207365676d656e74656420696e746f20736d616c6c6572207061636b657473206163636f7264696e6720746f20746865204d41502070726f63657373696e672066756e6374696f6e2eb7d3
